### Наивный байесовский классификатор для фильтрации спама в смс-сообщениях

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string, re
import itertools as it
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import TransformerMixin, BaseEstimator

In [2]:
# читаем данные
with open(r'.\SMSSpamCorpus01\english_big.txt', encoding='utf-8') as f:
    t = f.readlines()
    
target = [row.split(',')[-1].startswith('spam') for row in t]
text = [','.join(row.split(',')[:-1]) for row in t]

print(len(text), len(target))

train_ratio = 0.9
X_train, X_test, y_train, y_test = train_test_split(text, target, train_size=train_ratio, test_size=1-train_ratio)

1324 1324


#### Задача фильтрации спама и способ ее решения через формулу Байеса
Пусть есть некоторый набор текстов, разбитый на токены. Пусть каждому тексту соответствует вектор $\vec{x} = (x_1, x_2, x_3, .... x_n)$, где $x_i$ -- количество вхождений $i-го$ токена в текст, $n$ -- количество всех возможных токенов во всех текстах, "словарь". Пусть, кроме того, некоторым текстам проставлена метка <спам> или <не спам>, и требуется научиться ставить такие же метки для всех текстов.<br> Предполагается, что признаки $(x_1, x_2, x_3, .... x_n)$ распределены независимо друг от друга. (Из-за этого предположения классификатор и называется **_наивным_**, и, как ни странно, даже если это предположение не выполняется, классификатор, основанный на таком предположении, все равно работает хорошо!). Тогда, если известна каждая из вероятностей *P($\vec{x}$|спам)*, *P($\vec{x}$|не спам)*, *P(спам)*, *P(не спам)*,*P($\vec{x}$)*, каждый новый текст можно однозначно отнести либо к категории <спам>, либо к категории <не спам>, сравнив апостриорные вероятности, высчитанные по правилу Байеса: $$P(спам|\vec{x}) = \frac{P(спам) * P(\vec{x}|спам)}{P(\vec{x})} \,\,\,\,[1]$$ и $$P(не\, спам|\vec{x}) = \frac{P(не\, спам) * P(\vec{x}|не\, спам)}{P(\vec{x})}\,\,\,[2]$$ 

#### Воплощение в реальность
При реализации описанного подхода требуется решить две задачи: <br>1. Выбрать способ токенизации обучающих текстов. Класс CountVectorizer <br> 2. Выбрать способ оценки вероятностей *P($\vec{x}$|спам)*, *P($\vec{x}$|не спам)*, *P(спам)*, *P(не спам)*,*P($\vec{x}$)* в процессе обучения метода. Мы ведь не знаем эти вероятности точно, их требуется получить из размеченных данных. Для этих целей в sklearn есть три класса: BernoulliNB, GaussianNB, MultinomialNB

Признаки из текста можно вытащить тоже несколькими способами, по-разному настраивая CountVectorizer. Он работает следующим образом. При вызове .fit() происходит следующее:
    1. Входной текст делится на токены согласно параметру token_pattern
    2. Если какие-то из токенов есть в stop_words, они отбрасываются
    3. Потом из оставшихся токенов формируются энграммы с длинами, указанными в параметре ngram_range
    4. Если энграмма раньше не встречалась в словаре CountVectorizer'а, она добавляется в этот словарь
    5. Если частота, с которой энграмма встречается во всех документах, больше max_df или меньше min_df, она отбрасывается 
    6. Если в словаре в итоге оказывается больше "слов", чем указано в max_features, словарь обрезается так, чтобы остались только самые популярные слова
Итак, словарь сформирован. Дальше, при вызове .transform(), входной текст преобразуется в разреженную матрицу длиной в словарь, 
причем сколько раз соответствующая энграмма есть во входном тексте, такое значение будет в матрице на соответствующей этой энграмме месте. 
**Сведем интересующие нас параметры CountVectorizer в таблицу**


| Параметр      | возможные значения      | смысл параметра                                                                   | 
|---------------|-------------------------|-----------------------------------------------------------------------------------|
| token_pattern | '(?u)\\b\\w\\w+\\b' п.у.| "нарезает" входной текст на последовательности от пробела до пробела с длиной > 1 |
|               | '(?:(?!\d)\w){2,}'      | не учитывает цифры                                                                |
|               | '(?:(?!\d)\w)+'         | то же, допускает в качестве токенов последовательности длиной 1                   |
| ngram_range   | (1, 1) п.у.             |  просто берет все слова, т.н. bag of words-подход                                 |
|               | (1, 2)                  |  все слова плюс все сочетания из двух слов                                        |
|               | (2, 2)                  |  все сочетания из трех слов                                                       |
| stop_words    | None п.у.               | никакие токены не исключаются из входного текста при обработке                    |
|               | 'english'               | встроенный словарь английских стоп-слов                                           |
|               | stopwords               | стоп-слова библиотеки nltk                                                        |
| max_feautures | int, None п.у.          | максимальный возможный размер словаря, можно с ним поиграться                     |
| max_df, min_df| 1.0, 0.0  п.у.          | не учитывать токены, частоты которых не попадают в интервал  [min_df, max_df]     | 

После того, как мы сопоставили каждому входному тексту вектор токенов $\vec{x}$, нужно обучить байесовский классификатор, т.е. подобрать ему подходящие оценки вероятностей на основе имеющихся размеченных данных. Пакет sklearn предоставляет несколько классов, и отличаются они только способом оценки вероятностей: <br><br> 
    **1. BernoulliNB.**  Эта модель предполагает, что каждый токен либо входит в текст, либо не входит, причем количество вхождений не учитывается, но учитывается только сам факт вхождения. Кроме того, она предполагает, что каждый класс (т.е. условная вероятность *P($\vec{x}$|<метка>)* для всех возможных $\vec{x}$) распределен по Бернулли, а вероятность *P($\vec{x}$|спам)* = $\prod$(вероятность того, что токен i встретится в документе с меткой спам, если $x_i = 1 $) или (вероятность того, что токен i не встретится в документе с меткой спам, если $x_i=0$) <br>
    **2. MultinomialNB. ** Тут предполагается, что каждый класс распределен мультиномиально. *P($\vec{x}$|спам)* = $\prod$(вероятность того, что токен i встретится в документе с меткой спам $x_i$ раз)<br> 
    **3.  GaussianNB. **  А тут -- что каждый класс распределен нормально. Этот метод плохо работает с разреженными матрицами, потому что ему для построения оценок требуется много данных, а у нас для каждого входного текста почти все элементы вектора $\vec{x}$ зануляются. Поэтому этот подход здесь работает плохо, но мы, однако, попробуем применить его из исследовательских соображений.
    

Сведем настраиваеваемые параметры классификаторов в таблицу. 

| параметр  | описание параметра | возможные значения | BernoulliNB | GaussianNB | MultinomialNB |
|-----------|--------------------|--------------------|-------------|------------|---------------|
| alpha     | сглаживание| 0..1 п.у.    |     +       |     -      |      +        |
| binarize  | при каком значении частоты засчитывать слово как встречающееся в тексте                   | None, float, 0 п.у.|     +       |     -      |      -        |
| fit_prior | учитывать вероятности классов во входных данных или брать их из равномерного распределения| False, True п.у.   |     +       |     -      |      +        |
| class_prior| априорная вероятность каждого из классов | [a, b, c..], None | + | +* | + |


__* Для GaussianNB параметр называется priors__

#### Посмотрим сперва, какие результаты выдаст GaussianNB

In [7]:
class ToDenseTransformer(TransformerMixin, BaseEstimator):
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [8]:
gnb_pipe = Pipeline([('vector', CountVectorizer()),
                     ('todense', ToDenseTransformer()), 
                     ('gnb', GaussianNB())])
gnb_params = dict(
    vector__token_pattern = [r'(?u)\b\w\w+\b', r'(?:(?!\d)\w){2,}', r'(?:(?!\d)\w)+'],
    vector__ngram_range = [(1,1), (1,2), (2,2)],
    vector__stop_words = [None, 'english'],
    vector__max_features = [None],
    vector__max_df = (0.5, 0.75, 1.),
    vector__min_df = (0.005, 0.01),
    gnb__priors = [None] + [[0.243202416918429, 0.756797583081571]]
)
gnb_search = GridSearchCV(gnb_pipe, param_grid=gnb_params, verbose=1, scoring='f1')

In [9]:
gnb_search.fit(X_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed:  1.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocess...nizer=None, vocabulary=None)), ('todense', ToDenseTransformer()), ('gnb', GaussianNB(priors=None))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vector__stop_words': [None, 'english'], 'vector__min_df': (0.005, 0.01), 'vector__max_df': (0.5, 0.75, 1.0), 'vector__token_pattern': ['(?u)\\b\\w\\w+\\b', '(?:(?!\\d)\\w){2,}', '(?:(?!\\d)\\w)+'], 'gnb__priors': [None, [0.243202416918429, 0.756797583081571]], 'vector__ngram_range': [(1, 1), (1, 2), (2, 2)], 'vector__max_features': [None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=1)

In [11]:
gnb_search.best_estimator_, gnb_search.best_score_

(Pipeline(memory=None,
      steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.5, max_features=None, min_df=0.005,
         ngram_range=(1, 2), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?:(?!\\d)\\w){2,}',
         tokenizer=None, vocabulary=None)), ('todense', ToDenseTransformer()), ('gnb', GaussianNB(priors=None))]),
 0.97102640313019861)

In [13]:
# бернуллиевский наивный баесовский классификатор
bnb_pipe = Pipeline([('vector', CountVectorizer()), ('bnb', BernoulliNB())])
bnb_params = dict(
    vector__token_pattern = [r'(?u)\b\w\w+\b', r'(?:(?!\d)\w){2,}', r'(?:(?!\d)\w)+'],
    vector__ngram_range = [(1,1), (1,2), (2,2)],
    vector__stop_words = [None, 'english', stopwords.words('english')],
    vector__max_features = [None] + list(range(1000, 15000, 2000)),
    vector__max_df = (0.5, 0.75, 1.),
    vector__min_df = (0.005, 0.01),
    bnb__binarize = np.arange(0, 0.5, 0.1),
    bnb__alpha = [0.5, 1]
)
bnb_search = GridSearchCV(bnb_pipe, param_grid=bnb_params, verbose=1,n_jobs=4, scoring='f1')

In [14]:
%%time
bnb_search.fit(X_train, y_train)

Fitting 3 folds for each of 12960 candidates, totalling 38880 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   11.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   24.3s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   42.5s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed:  3.5min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed:  6.0min
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed:  7.0min
[Parallel(n_jobs=4)]: Done 9792 tasks      | elapsed:  8.0min
[Parallel(n_jobs=4)]: Done 11242 tasks      | elapsed:  9.2min
[Parallel(n_jobs=4)]: Done 12792 tasks      | elapsed: 10.3min
[Parallel(

Wall time: 30min 43s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocess...vocabulary=None)), ('bnb', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vector__stop_words': [None, 'english', ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', ...or__min_df': (0.005, 0.01), 'bnb__alpha': [0.5, 1], 'vector__ngram_range': [(1, 1), (1, 2), (2, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1',

In [15]:
# полиномиальный наивный баесовский классификатор
mnb_pipe = Pipeline([('vector', CountVectorizer()), ('mnb', MultinomialNB())])
mnb_params = dict(
    vector__token_pattern = [r'(?u)\b\w\w+\b', r'(?:(?!\d)\w){2,}', r'(?:(?!\d)\w)+'],
    vector__ngram_range = [(1,1), (1,2), (2,2)],
    vector__stop_words = [None, 'english', stopwords.words('english')],
    vector__max_features = [None] + list(range(1000, 15000, 2000)),
    vector__max_df = (0.5, 0.75, 1.),
    vector__min_df = (0.005, 0.01),
    mnb__alpha = [0.5, 1]
)
mnb_search = GridSearchCV(mnb_pipe, param_grid=mnb_params, verbose=1, n_jobs=4, scoring='f1')

In [16]:
%%time
mnb_search.fit(X_train, y_train)

Fitting 3 folds for each of 2592 candidates, totalling 7776 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   10.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   21.8s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   37.7s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:   58.4s
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed:  5.5min
[Parallel(n_jobs=4)]: Done 7776 out of 7776 | elapsed:  5.9min finished


Wall time: 5min 54s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocess...nizer=None, vocabulary=None)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vector__stop_words': [None, 'english', ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', ..._min_df': (0.005, 0.01), 'vector__max_features': [None, 1000, 3000, 5000, 7000, 9000, 11000, 13000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1',

In [17]:
bnb_search.best_estimator_, bnb_search.best_score_

(Pipeline(memory=None,
      steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.5, max_features=None, min_df=0.005,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None, vocabulary=None)), ('bnb', BernoulliNB(alpha=0.5, binarize=0.0, class_prior=None, fit_prior=True))]),
 0.98459731372794024)

In [18]:
mnb_search.best_estimator_, mnb_search.best_score_

(Pipeline(memory=None,
      steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.5, max_features=None, min_df=0.005,
         ngram_range=(1, 2), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?:(?!\\d)\\w){2,}',
         tokenizer=None, vocabulary=None)), ('mnb', MultinomialNB(alpha=1, class_prior=None, fit_prior=True))]),
 0.98147145986341955)

In [44]:
# посмотрим теперь на поведение этих классификаторов на отлженной тестовой выборке
names = ['multinomial', 'bernoulli', 'gaussian']
classifiers = [mnb_search.best_estimator_, bnb_search.best_estimator_, gnb_search.best_estimator_]
for n, clf in zip(names, classifiers):
    print('-----------', n, '-------------')
    print(classification_report(y_test, clf.predict(X_test), digits=5))

----------- multinomial -------------
             precision    recall  f1-score   support

      False    0.99065   0.98148   0.98605       108
       True    0.92308   0.96000   0.94118        25

avg / total    0.97795   0.97744   0.97761       133

----------- bernoulli -------------
             precision    recall  f1-score   support

      False    0.98165   0.99074   0.98618       108
       True    0.95833   0.92000   0.93878        25

avg / total    0.97727   0.97744   0.97727       133

----------- gaussian -------------
             precision    recall  f1-score   support

      False    1.00000   0.96296   0.98113       108
       True    0.86207   1.00000   0.92593        25

avg / total    0.97407   0.96992   0.97075       133



In [51]:
# а теперь попробуем еще и на другом датасете проверить эти фильтры
with open(r'.\SMSSpamCorpus01\english.txt', encoding='utf-8') as f:
    another_text = f.readlines()
_X = [''.join(text.split(',')[:-1]) for text in another_text]
_y = [text.split(',')[-1]=='spam\n' for text in another_text]
Counter(_y) # как видим, тут уже совсем другое распределение, это может здорово сказаться на надежности классификаторов

Counter({False: 1003, True: 82})

In [56]:
for n, clf in zip(names, classifiers):
    print('-----------', n, '-------------')
    print(classification_report(_y, clf.predict(_X), digits=5))

----------- multinomial -------------
             precision    recall  f1-score   support

      False    0.99800   0.99402   0.99600      1003
       True    0.93023   0.97561   0.95238        82

avg / total    0.99288   0.99263   0.99271      1085

----------- bernoulli -------------
             precision    recall  f1-score   support

      False    0.99701   0.99801   0.99751      1003
       True    0.97531   0.96341   0.96933        82

avg / total    0.99537   0.99539   0.99538      1085

----------- gaussian -------------
             precision    recall  f1-score   support

      False    0.99800   0.99402   0.99600      1003
       True    0.93023   0.97561   0.95238        82

avg / total    0.99288   0.99263   0.99271      1085



**В худшем случае (бернуллиевский наивный байес) классификатор пропускает 3,3% спама**. Кажется, это вполне неплохо. Удивительно, что цифры для мультиноминального и гауссовского классификаторов получились одинаковые с точностью до 5 знаков после запятой.

In [67]:
# попробуем теперь еще на одном датасете
with open(r'.\sms-spam-collection-dataset\spam.csv') as f:
    another_data = f.readlines()[1:]
yy = [data.split(',',maxsplit=1)[0]=='spam' for data in another_data]
xx = [data.split(',',maxsplit=1)[1] for data in another_data]

In [70]:
Counter(yy) 

Counter({False: 4827, True: 747})

In [71]:
for n, clf in zip(names, classifiers):
    print('-----------', n, '-------------')
    print(classification_report(yy, clf.predict(xx), digits=5))

----------- multinomial -------------
             precision    recall  f1-score   support

      False    0.98970   0.91589   0.95137      4827
       True    0.63324   0.93842   0.75620       747

avg / total    0.94193   0.91891   0.92521      5574

----------- bernoulli -------------
             precision    recall  f1-score   support

      False    0.98651   0.96996   0.97817      4827
       True    0.82488   0.91432   0.86730       747

avg / total    0.96485   0.96250   0.96331      5574

----------- gaussian -------------
             precision    recall  f1-score   support

      False    0.98736   0.92231   0.95373      4827
       True    0.64789   0.92369   0.76159       747

avg / total    0.94186   0.92250   0.92798      5574



#### Видно, что несмотря на полностью новый набор данных, классификаторы еще неплохо работают,
хоть и записывают много лишних сообщений в спам. Но, если в общем, у нас **получилось обучить все три модели с неплохой точностью.**

#### Что еще можно улучшить в моделях или дополнительно поисследовать?
1. Cтоит еще раз обучить модели на новых данных, скомбинированных их нескольких датасетов.
2. Стоит еще тщательнее подойти к подбору параметров классификаторов, в частности, стоит попробовать более качественно подобрать max_df и min_df для BernoulliNB с MultinomialNB, а для GaussianNB попробовать триграммы. 
3. Поскольку отношение текстов в категориях разнится от датасета к датасету, стоит попробовать запретить классификаторам использовать чистые вероятности _P(<спам>)_ и _P(<не спам>)_, выставив fit_prior=False и вручную задав class_prior

Может быть также, что идея создать универсальный спам-фильтр сама по себе не может быть решена с хорошей точностью, т.к. слишком многое во входящем потоке текстов зависит от природы этих текстов, и универсальный спам-фильтр должен учитывать эту природу, чтобы быть качественным.